<a href="https://colab.research.google.com/github/riphunter7001x/fine-tuning-smalll-language-model-on-disease-dataset/blob/main/small_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

In [17]:
from datasets import load_dataset,DatasetDict,Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time

In [18]:
# load daatset
data_sample = load_dataset("QuyenAnhDE/Diseases_Symptoms")

Repo card metadata block was not found. Setting CardData to empty.


In [19]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [20]:
updated_date = [{"Name":item["Name"],"Symptoms":item["Symptoms"]} for item in data_sample["train"]]

In [21]:
df = pd.DataFrame(updated_date)

In [22]:
df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [23]:
# just extract the symptoms
df["Symptoms"] = df["Symptoms"].apply(lambda x: " , ".join(x.split(", ")))

In [24]:
df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations , Sweating , Trembling , Shortnes..."
1,Vocal cord polyp,"Hoarseness , Vocal Changes , Vocal Fatigue"
2,Turner syndrome,"Short stature , Gonadal dysgenesis , Webbed ne..."
3,Cryptorchidism,"Absence or undescended testicle(s) , empty scr..."
4,Ethylene glycol poisoning-1,"Nausea , vomiting , abdominal pain , General m..."


In [25]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,random_split


In [26]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [27]:
device

device(type='cuda')

In [28]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [29]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [30]:
BATCH_SIZE = 8

In [31]:
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling , pain , dry mouth , bad taste"
freq,3,3


In [48]:
# Dataset Prep
class LanguageDataset(Dataset):
    """
    An extension of the Dataset object to:
      - Make training loop cleaner
      - Make ingestion easier from pandas df's
    """
    def __init__(self, df, tokenizer):
        self.labels = df.columns
        self.data = df.to_dict(orient='records')
        self.tokenizer = tokenizer
        x = self.fittest_max_length(df)  # Fix here
        self.max_length = x

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx][self.labels[0]]
        y = self.data[idx][self.labels[1]]
        text = f"{x} | {y}"
        tokens = self.tokenizer.encode_plus(text, return_tensors='pt', max_length=128, padding='max_length', truncation=True)
        return tokens

    def fittest_max_length(self, df):  # Fix here
        """
        Smallest power of two larger than the longest term in the data set.
        Important to set up max length to speed training time.
        """
        max_length = max(len(max(df[self.labels[0]], key=len)), len(max(df[self.labels[1]], key=len)))
        x = 2
        while x < max_length: x = x * 2
        return x


In [49]:
data_sample = LanguageDataset(df,tokenizer)

In [50]:
data_sample

In [51]:
train_size = int(0.8 * len(data_sample))
val_size = len(data_sample) - train_size
train_data, val_data = random_split(data_sample, [train_size, val_size])

In [52]:
train_data

In [53]:
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE)

In [54]:
num_epochs = 8

In [68]:
batch_size = BATCH_SIZE
model_name = 'distilgpt2'
gpu = 0

In [71]:

# Set the learning rate and loss function
## CrossEntropyLoss measures how close answers to the truth.
## More punishing for high confidence wrong answers
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token

In [72]:
results = pd.DataFrame(columns=['epoch', 'transformer',"batch_size","gpu","training_loss" , 'validation_loss',"epoch_duration"])


In [76]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [80]:
# The training loop
for epoch in range(num_epochs):
    start_time = time.time()  # Start the timer for the epoch

    # Training
    ## This line tells the model we're in 'learning mode'
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = batch['input_ids'].squeeze(1).to(device)
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({'Training Loss': loss.item()})
        epoch_training_loss += loss.item()
    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    # Validation
    ## This line below tells the model to 'stop learning'
    model.eval()
    epoch_validation_loss = 0
    total_loss = 0
    valid_iterator = tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            total_loss += loss
            valid_iterator.set_postfix({'Validation Loss': loss.item()})
            epoch_validation_loss += loss.item()

    avg_epoch_validation_loss = epoch_validation_loss / len(val_loader)

    end_time = time.time()  # End the timer for the epoch
    epoch_duration_sec = end_time - start_time  # Calculate the duration in seconds

    new_row = {'transformer': model_name,
               'batch_size': batch_size,
               'gpu': gpu,
               'epoch': epoch+1,
               'training_loss': avg_epoch_training_loss,
               'validation_loss': avg_epoch_validation_loss,
               'epoch_duration_sec': epoch_duration_sec}  # Add epoch_duration to the dataframe

    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(val_loader)}")

Training Epoch 1/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.87it/s, Training Loss=0.347]
Validation Epoch 1/8: 100%|██████████| 10/10 [00:00<00:00, 16.80it/s, Validation Loss=0.68]


Epoch: 1, Validation Loss: 0.7090097665786743


Training Epoch 2/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.78it/s, Training Loss=0.184]
Validation Epoch 2/8: 100%|██████████| 10/10 [00:00<00:00, 15.94it/s, Validation Loss=0.725]


Epoch: 2, Validation Loss: 0.7536070942878723


Training Epoch 3/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.70it/s, Training Loss=0.237]
Validation Epoch 3/8: 100%|██████████| 10/10 [00:00<00:00, 16.20it/s, Validation Loss=0.748]


...

Epoch: 6, Validation Loss: 0.9297280311584473


Training Epoch 7/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.89it/s, Training Loss=0.111]
Validation Epoch 7/8: 100%|██████████| 10/10 [00:00<00:00, 16.74it/s, Validation Loss=0.928]


Epoch: 7, Validation Loss: 0.9607395529747009


Training Epoch 8/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:08<00:00,  4.90it/s, Training Loss=0.0947]
Validation Epoch 8/8: 100%|██████████| 10/10 [00:00<00:00, 16.69it/s, Validation Loss=0.976]

Epoch: 8, Validation Loss: 1.0031393766403198


In [81]:
input_str = "kidney failure"

In [82]:
input_ids = tokenizer.encode(input_str,return_tensors= "pt").to(device)

In [83]:
input_ids

tensor([[38439,  1681,  5287]], device='cuda:0')

In [91]:
output = model.generate(
    input_ids,
    max_length =128,
    num_return_sequences = 1,
    do_sample=True,
    temperature=0.5,
    top_k = 8,
    top_p = 0.95,
    repetition_penalty = 1.2,
    # no_repeat_ngram_size = 2,
    # early_stopping = True,
    # pad_token_id = tokenizer.eos_token_id
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [92]:
decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

In [93]:
decoded_output

'kidney failure | Decreased urine output, fluid retention, fatigue, shortness of breath, nausea, confusion, chest pain or discomfort, seizures'

In [94]:
# save model
torch.save(model ,"SmallDiseaseLM.pt")

In [95]:
torch.save(model ,"/content/drive/MyDrive/fine_tuning/SmallDiseaseLM.pt")